# Código y trabajo

Parte 2
-----


**Restricción:** `No librerías, excepto numpy`

In [1]:
import numpy as np

### Enunciado
* Escriba un programa que permita entrenar una red FF con 1 capa escondida (H neuronas) y O neuronas de salida, **sin usar librerías**, excepto eventualmente *numpy* para implementar operaciones básicas de álgebra lineal. Por simplicidad, asuma que todas las neuronas implementan una función de activación diferenciable y que la función de error (*loss function*) también lo es. Especiﬁque explícitamente las funciones anteriores, así como sus gradientes. Escriba funciones para: 
    1. Dar valores iniciales a los pesos de la red
    2. Implementar el forward pass
    3. Implementar el backward pass
    4. Implementar la rutina principal de entrenamiento, adoptando, por simplicidad, la variante cíclica de SGD (un ejemplo a la vez, pero iterando cíclicamente sobre el conjunto de entrenamiento) con una tasa de aprendizaje y número de ciclos ﬁjos (epochs).
----------
*  Escriba una función que permita hacer predicciones mediante una red FF con 1 capa escondida (H neuronas) y O neuronas de salida, sin usar librerías, excepto eventualmente numpy. Escriba una función vectorizada que implemente el **forward pass** sobre un conjunto de *n<sub>test</sub>* ejemplos.

----------

* Demuestre que sus programas funcionan en un problema de clasiﬁcación, eligiendo funciones de error y de activación apropiadas. Para esto utilice el dataset seeds, disponible en [UCI](http://archive.ics.uci.edu/ml/) y correspondiente a la clasiﬁcación de distintos tipos de semillas (3 clases) (recuerde que usualmente es conveniente normalizar los datos antes de trabajar con el modelo).

```python
from sklearn.preprocessing import StandardScaler
import pandas as pd
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt'
df = pd.read_csv(url, sep=r'\s+',header=None)
X_train = df.ix[:,0:6]
y_train = df.ix[:,7]
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

```
Para evaluar los resultados, construya un gráﬁco correspondiente al error de clasiﬁcación versus número de *epochs*, utilizando sólo el conjunto de entrenamiento (el objetivo de esta sección es familiarizarse con el algoritmo BP, no encontrar la mejor red). Graﬁque también la evolución de la función objetivo utilizada para el entrenamiento.

----------

* Construya, sin usar librerías, excepto eventualmente numpy para implementar operaciones básicas de álgebra lineal, una variante de su programa anterior que entrene la red utilizando *weight-decay*.
